In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
#tf.enable_eager_execution()

In [2]:
from tensorflow import keras
from tensorflow.keras.layers import LSTM,Dropout,Activation,Dense
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [94]:
tf.executing_eagerly()

True

In [3]:
path=tf.keras.utils.get_file('shakespeare','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text=open(path,'r').read()

In [4]:
print(text[0:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [4]:
text=text.lower()
text=''.join(c for c in text if not c.isdigit())
len(text)

1115367

In [5]:
vocab=sorted(list(set(text)))
len(vocab)

38

In [6]:
vocab

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [7]:
char_to_int={c:i for i,c in enumerate(vocab)}

In [8]:
char_to_int['a']

12

In [9]:
int_to_char=np.array(vocab)

In [10]:
int_to_char[12]

'a'

In [11]:
encoded_text=np.array([char_to_int[c] for c in text])

In [12]:
encoded_text.shape

(1115367,)

In [13]:
encoded_text[0:100]

array([17, 20, 29, 30, 31,  1, 14, 20, 31, 20, 37, 16, 25,  9,  0, 13, 16,
       17, 26, 29, 16,  1, 34, 16,  1, 27, 29, 26, 14, 16, 16, 15,  1, 12,
       25, 36,  1, 17, 32, 29, 31, 19, 16, 29,  6,  1, 19, 16, 12, 29,  1,
       24, 16,  1, 30, 27, 16, 12, 22,  8,  0,  0, 12, 23, 23,  9,  0, 30,
       27, 16, 12, 22,  6,  1, 30, 27, 16, 12, 22,  8,  0,  0, 17, 20, 29,
       30, 31,  1, 14, 20, 31, 20, 37, 16, 25,  9,  0, 36, 26, 32])

In [14]:
text[0:100]

'first citizen:\nbefore we proceed any further, hear me speak.\n\nall:\nspeak, speak.\n\nfirst citizen:\nyou'

In [15]:
seq_len=100

In [16]:
total_num_seq=len(text) // (seq_len+1)
print(total_num_seq)

11043


In [17]:
char_dataset=tf.data.Dataset.from_tensor_slices(encoded_text)

In [18]:
type(char_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [19]:
for item in char_dataset.take(100):
    print(int_to_char[item.numpy()])

f
i
r
s
t
 
c
i
t
i
z
e
n
:


b
e
f
o
r
e
 
w
e
 
p
r
o
c
e
e
d
 
a
n
y
 
f
u
r
t
h
e
r
,
 
h
e
a
r
 
m
e
 
s
p
e
a
k
.




a
l
l
:


s
p
e
a
k
,
 
s
p
e
a
k
.




f
i
r
s
t
 
c
i
t
i
z
e
n
:


y
o
u


In [20]:
sequences=char_dataset.batch(batch_size=seq_len+1,drop_remainder=True)

In [21]:
def create_seq_target(seq):
    input_text=seq[:-1]
    output_text=seq[1:]
    return input_text,output_text    

In [22]:
dataset=sequences.map(create_seq_target)

In [23]:
for input_txt,output_txt in dataset.take(1):
    print(input_txt.numpy())
    print("\n")
    print("".join(int_to_char[input_txt.numpy()]))
    print(output_txt.numpy())
    print("\n")
    print("".join(int_to_char[output_txt.numpy()]))

[17 20 29 30 31  1 14 20 31 20 37 16 25  9  0 13 16 17 26 29 16  1 34 16
  1 27 29 26 14 16 16 15  1 12 25 36  1 17 32 29 31 19 16 29  6  1 19 16
 12 29  1 24 16  1 30 27 16 12 22  8  0  0 12 23 23  9  0 30 27 16 12 22
  6  1 30 27 16 12 22  8  0  0 17 20 29 30 31  1 14 20 31 20 37 16 25  9
  0 36 26 32]


first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you
[20 29 30 31  1 14 20 31 20 37 16 25  9  0 13 16 17 26 29 16  1 34 16  1
 27 29 26 14 16 16 15  1 12 25 36  1 17 32 29 31 19 16 29  6  1 19 16 12
 29  1 24 16  1 30 27 16 12 22  8  0  0 12 23 23  9  0 30 27 16 12 22  6
  1 30 27 16 12 22  8  0  0 17 20 29 30 31  1 14 20 31 20 37 16 25  9  0
 36 26 32  1]


irst citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you 


In [24]:
buffer_size=10000
batch=64
dataset=dataset.shuffle(buffer_size).batch(batch_size=64,drop_remainder=True)

In [25]:
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [26]:
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [27]:
vocab_size=len(vocab)

In [28]:
embed_size=56

In [29]:
units=1024

In [30]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [31]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(
      y_true=labels,
      y_pred=logits,
      from_logits=True
    )
    

In [32]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding,GRU
def build_model(output_dim,vocab_size,batch_size,units):
    model = Sequential()
    model.add(Embedding(vocab_size,output_dim,batch_input_shape=(batch_size,None)))
    
    model.add(LSTM(
      units=units,
      return_sequences=True,
      stateful=True,recurrent_initializer='glorot_uniform'
      
    ))
                                 
    model.add(Dense(vocab_size))
    
    model.compile(loss=loss,optimizer=Adam(0.001),metrics=['sparse_categorical_accuracy'])
    
    return model

In [33]:
model=build_model(embed_size,vocab_size,batch,units)

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 56)            2128      
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          4427776   
_________________________________________________________________
dense (Dense)                (64, None, 38)            38950     
Total params: 4,468,854
Trainable params: 4,468,854
Non-trainable params: 0
_________________________________________________________________


In [35]:
for input_example_batch,target_example_batch in dataset.take(1):
      batch_example_predictions=model(input_example_batch)

In [36]:
input_example_batch.shape

TensorShape([64, 100])

In [37]:
input_example_batch[0]

<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([ 9,  0, 36, 16, 31,  1, 23, 26, 26, 22, 30,  1, 19, 16,  1, 23, 20,
       22, 16,  1, 12,  1, 22, 20, 25, 18,  9,  1, 13, 16, 19, 26, 23, 15,
        6,  1, 19, 20, 30,  1, 16, 36, 16,  6,  0, 12, 30,  1, 13, 29, 20,
       18, 19, 31,  1, 12, 30,  1, 20, 30,  1, 31, 19, 16,  1, 16, 12, 18,
       23, 16,  5, 30,  6,  1, 23, 20, 18, 19, 31, 16, 25, 30,  1, 17, 26,
       29, 31, 19,  0, 14, 26, 25, 31, 29, 26, 23, 23, 20, 25, 18])>

In [38]:
batch_example_predictions.shape

TensorShape([64, 100, 38])

In [39]:
target_example_batch[0]

<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([ 0, 36, 16, 31,  1, 23, 26, 26, 22, 30,  1, 19, 16,  1, 23, 20, 22,
       16,  1, 12,  1, 22, 20, 25, 18,  9,  1, 13, 16, 19, 26, 23, 15,  6,
        1, 19, 20, 30,  1, 16, 36, 16,  6,  0, 12, 30,  1, 13, 29, 20, 18,
       19, 31,  1, 12, 30,  1, 20, 30,  1, 31, 19, 16,  1, 16, 12, 18, 23,
       16,  5, 30,  6,  1, 23, 20, 18, 19, 31, 16, 25, 30,  1, 17, 26, 29,
       31, 19,  0, 14, 26, 25, 31, 29, 26, 23, 23, 20, 25, 18,  1])>

In [40]:
#shape(batch_size,input_length,vocab_size)
batch_example_predictions[0]

<tf.Tensor: shape=(100, 38), dtype=float32, numpy=
array([[ 0.00187001,  0.00031166, -0.00051023, ...,  0.00079601,
         0.00208398, -0.00043685],
       [ 0.00228901, -0.00033881, -0.00190261, ..., -0.00022432,
         0.00131825, -0.000849  ],
       [ 0.00532284,  0.00218869, -0.00362678, ...,  0.00170633,
         0.00149027,  0.00127491],
       ...,
       [ 0.00147808, -0.00484045,  0.00143492, ..., -0.00526975,
         0.00066018,  0.00122667],
       [ 0.00181574, -0.00329164,  0.00406523, ..., -0.00401777,
        -0.00036749, -0.0017116 ],
       [ 0.00114387, -0.00479589,  0.00379273, ..., -0.00564774,
        -0.00025505,  0.00045959]], dtype=float32)>

In [41]:
random_samples=tf.random.categorical(batch_example_predictions[0],num_samples=1)

In [42]:
print(tf.random.categorical(batch_example_predictions[0],num_samples=1)[-1,0].numpy())

23


In [43]:
random_samples

<tf.Tensor: shape=(100, 1), dtype=int64, numpy=
array([[30],
       [24],
       [32],
       [37],
       [16],
       [ 1],
       [17],
       [22],
       [ 5],
       [26],
       [ 9],
       [32],
       [24],
       [32],
       [ 3],
       [28],
       [ 9],
       [30],
       [27],
       [31],
       [ 0],
       [ 2],
       [26],
       [32],
       [35],
       [18],
       [32],
       [18],
       [34],
       [ 1],
       [33],
       [32],
       [20],
       [14],
       [ 9],
       [32],
       [32],
       [ 2],
       [ 1],
       [ 2],
       [17],
       [15],
       [36],
       [25],
       [ 2],
       [34],
       [ 0],
       [15],
       [ 8],
       [34],
       [ 0],
       [ 3],
       [28],
       [35],
       [28],
       [30],
       [33],
       [11],
       [13],
       [ 4],
       [15],
       [ 7],
       [10],
       [31],
       [ 5],
       [ 7],
       [34],
       [ 7],
       [ 1],
       [28],
       [ 0],
       [20],
       [10],
   

In [44]:
batch_example_predictions[0][-1,0].numpy()

0.0011438701

In [45]:
random_samples.shape

TensorShape([100, 1])

In [46]:
random_samples=tf.squeeze(random_samples)

In [47]:
random_samples.shape

TensorShape([100])

In [48]:
int_to_char[random_samples]

array(['s', 'm', 'u', 'z', 'e', ' ', 'f', 'k', "'", 'o', ':', 'u', 'm',
       'u', '$', 'q', ':', 's', 'p', 't', '\n', '!', 'o', 'u', 'x', 'g',
       'u', 'g', 'w', ' ', 'v', 'u', 'i', 'c', ':', 'u', 'u', '!', ' ',
       '!', 'f', 'd', 'y', 'n', '!', 'w', '\n', 'd', '.', 'w', '\n', '$',
       'q', 'x', 'q', 's', 'v', '?', 'b', '&', 'd', '-', ';', 't', "'",
       '-', 'w', '-', ' ', 'q', '\n', 'i', ';', 'v', 'q', 'o', 'x', 'p',
       'm', 'h', 'w', ':', 'n', 'u', '?', '?', ',', 'e', 'k', 'i', ':',
       '&', 'm', 'c', 'g', 'l', 'l', 'r', "'", 'a'], dtype='<U1')

In [49]:
loss(target_example_batch,batch_example_predictions)[0]

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([3.635406 , 3.636222 , 3.6379583, 3.6362448, 3.6365352, 3.636787 ,
       3.6393318, 3.6380754, 3.6332123, 3.6379316, 3.640997 , 3.6362758,
       3.6351805, 3.6436336, 3.6376798, 3.6375248, 3.6353955, 3.637597 ,
       3.639986 , 3.6341379, 3.643597 , 3.6367545, 3.6370041, 3.6384733,
       3.645325 , 3.6398633, 3.640744 , 3.6373827, 3.6389246, 3.63321  ,
       3.6398818, 3.6369796, 3.632824 , 3.6402133, 3.6440184, 3.6347215,
       3.6364026, 3.6315572, 3.6407158, 3.636084 , 3.6342268, 3.6356025,
       3.6373181, 3.637705 , 3.6407495, 3.63679  , 3.6379685, 3.6357782,
       3.6353822, 3.6369388, 3.6421204, 3.6343837, 3.6350257, 3.6438131,
       3.6339073, 3.634898 , 3.6403804, 3.6379185, 3.6309266, 3.638724 ,
       3.6337855, 3.6330333, 3.635371 , 3.6412673, 3.6362453, 3.63388  ,
       3.644438 , 3.6354144, 3.6329114, 3.6375437, 3.6342328, 3.6384807,
       3.6393607, 3.63931  , 3.6373072, 3.642086 , 3.6313818, 3.6346319,
   

In [50]:
import os
lstm_dir_checkpoints= './training_checkpoints_LSTM'
#checkpoint_prefix = os.path.join(lstm_dir_checkpoints, "checkpt_{epoch}") #name
#checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

In [187]:
history = model.fit(dataset, epochs=40, callbacks=[checkpoint_callback])

Epoch 1/40
172/172 [==============================] - 464s 3s/step - loss: 1.9622 - sparse_categorical_accuracy: 0.4191
Epoch 2/40
172/172 [==============================] - 473s 3s/step - loss: 1.6964 - sparse_categorical_accuracy: 0.4919
Epoch 3/40
172/172 [==============================] - 462s 3s/step - loss: 1.5451 - sparse_categorical_accuracy: 0.5337
Epoch 4/40
172/172 [==============================] - 467s 3s/step - loss: 1.4518 - sparse_categorical_accuracy: 0.5580
Epoch 5/40
172/172 [==============================] - 466s 3s/step - loss: 1.3864 - sparse_categorical_accuracy: 0.5754
Epoch 6/40
172/172 [==============================] - 468s 3s/step - loss: 1.3343 - sparse_categorical_accuracy: 0.5887
Epoch 7/40
172/172 [==============================] - 476s 3s/step - loss: 1.2899 - sparse_categorical_accuracy: 0.6004
Epoch 8/40
172/172 [==============================] - 470s 3s/step - loss: 1.2481 - sparse_categorical_accuracy: 0.6127
Epoch 9/40
172/172 [====================

In [51]:
tf.train.latest_checkpoint(lstm_dir_checkpoints)

'./training_checkpoints_LSTM\\checkpt_40'

In [52]:
lstm_model = build_model(embed_size,vocab_size,1,units)
lstm_model.load_weights(tf.train.latest_checkpoint(lstm_dir_checkpoints))
lstm_model.build(tf.TensorShape([1, None]))

lstm_model.summary() 


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.embeddings.Embedding object at 0x00000206A3B42388> and <tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x00000206A3B3C088>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x00000206A3B3C088> and <tensorflow.python.keras.layers.core.Dense object at 0x00000206FA30D9C8>).
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 56)             2128      
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           4427776   
_________________________________________________________________
dense_1 (Dense)              (1, None, 38)             38950     
Total params: 4,468,854
Trainable params: 4,468,854
Non-trainable params

In [73]:
def generate_text(model, start_string):
    num_generate = 200 #Number of characters to be generated

    input_eval = [char_to_int[s] for s in start_string] #vectorising input
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.1
    
    k=0
    # Here batch size == 1
    model.reset_states()
    print(input_eval.shape)
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(int_to_char[predicted_id])
        #start_string=start_string+int_to_char[predicted_id]
        #input_eval = [char_to_int[s] for s in start_string[k+1:]] #vectorising input
        #print(len(input_eval))
        input_eval = tf.expand_dims(input_eval, 0)
        #k+=1
        

    return (start_string+''.join(text_generated))

In [74]:
lstm_test = input("Enter your starting string: ")
print(generate_text(lstm_model, start_string=lstm_test))

Enter your starting string: first
(1, 5)
first keep the people, which
are made the conducts of this place of constable,
and fear'd as ourselves to see 'emeand he shall be frief:
the more i bid thee dead, and i emplay
the thumber is not hot i see 
